<a href="https://colab.research.google.com/github/kartikgandhi/Cross-Project-Hard-Voting-Edited-Dataset/blob/main/Cross_Project_HV_train_mina%2Btest_greenstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [337]:
from numpy import mean
from numpy import std

In [338]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [339]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [340]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [341]:
#file to initalize the weights from
weights = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-mina data 2.0.7-2.0.8 - mina data 2.0.7-2.0.8.csv")


In [342]:
weights['change'] = weights['change'].map({'yes': 1, 'no': 0})

In [343]:
weights.shape

(610, 8)

In [344]:
weights.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,1,0,53,15,0,100,4,0
1,10,0,391,74,2,66,12,0
2,25,2,194,225,0,98,181,0
3,4,0,14,47,0,66,1,0
4,34,0,53,1049,0,0,40,0


In [345]:
X = weights.drop('change', axis=1)
y = weights['change']

In [346]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=1)

In [347]:
# get a list of base models
def get_models():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))
	return models

In [348]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores

In [349]:
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)

[0.8415841584158416, 0.8415841584158416, 0.8415841584158416]


In [350]:
data = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-greenstone 3.0.7 - greenstone 3.0.7.csv")

In [351]:
data['change'] = data['change'].map({'YES': 1, 'NO': 0})

In [352]:
data.shape

(601, 8)

In [353]:
data.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,4,0,34,189,1,78,24,1
1,6,0,5,133,1,50,20,1
2,7,0,60,2124,1,0,355,1
3,21,0,15,145,1,85,17,1
4,10,0,13,721,1,91,27,1


In [354]:
X = data.drop('change', axis=1)
y = data['change']

In [355]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [356]:
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='hard', weights=scores)
# fit the ensemble on the training dataset
ensemble=ensemble.fit(X_train, y_train)
# make predictions on test set
yhat = ensemble.predict(X_test)

In [357]:
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Weighted Avg Accuracy: %.3f' % (score*100))

Weighted Avg Accuracy: 85.124


In [358]:
print(yhat)

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]


In [359]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,yhat)
print(cm)

[[102   0]
 [ 18   1]]


In [360]:
# predict probabilities
# from sklearn.metrics import roc_auc_score
# pred_prob = ensemble.predict_proba(X_test)
# auc_score = roc_auc_score(y_test, pred_prob[:,1])
# print("Area Under Curve=")
# print(auc_score)

In [361]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,yhat)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.2115108557622118


In [362]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, yhat)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5263157894736842


In [363]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, yhat)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.22941573387056174


In [364]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       102
           1       1.00      0.05      0.10        19

    accuracy                           0.85       121
   macro avg       0.93      0.53      0.51       121
weighted avg       0.87      0.85      0.79       121

